In [1]:
# -----------------------------------------"Final UI"-----------------------------------------

import tkinter as tk
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from transformers import pipeline
import random

jokes = [
    "Why don't skeletons fight each other? They don't have the guts!",
    "I told my computer I needed a break, and now it won’t stop sending me Kit-Kats.",
    "What did one ocean say to the other ocean? Nothing, they just waved.",
    "Why don’t orphans play hide and seek? Because no one will look for them."
]

def tell_joke():
    return random.choice(jokes)

# --> Loading fine-tuned model
tokenizer = BlenderbotTokenizer.from_pretrained("C:/Users/HP/OneDrive/Documents/NLP_Project/Implementation_Folder/App/fine_tuned_blenderbot")
model = BlenderbotForConditionalGeneration.from_pretrained("C:/Users/HP/OneDrive/Documents/NLP_Project/Implementation_Folder/App/fine_tuned_blenderbot")

# --> Generate chatbot response
def generate_response(user_input):
    inputs = tokenizer([user_input], return_tensors="pt", padding=True, truncation=True)
    output = model.generate(inputs["input_ids"], max_length=100, num_beams=5, no_repeat_ngram_size=2)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# --> Loading sentiment and emotion models
sentiment_analyzer = pipeline("sentiment-analysis")
emotion_analyzer = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

# --> Analyze sentiment and emotion
def analyze_sentiment_and_emotion(text):
    sentiment = sentiment_analyzer(text)[0]
    emotion = emotion_analyzer(text)[0]
    return sentiment, emotion

# -----------------------------------------"Main GUI"-----------------------------------------
from time import strftime

# --> Function to update the Time
def update_time(label):
    current_time = strftime('%I:%M:%S %p') 
    label.config(text=current_time)
    label.after(1000, update_time, label)

# --> Main window setup
root = tk.Tk()
root.title("Friendly Chat-Bot")
root.geometry("400x500")
root.resizable(False, False)
root.configure(bg="#81c6c1")

# --> Time Positioning
time_label = tk.Label(
    root,
    font=("Montserrat", 24, "bold"),
    fg="#2d2d2d",
    bg="#81c6c1",
    highlightthickness=0
)
time_label.place(relx=0.5, rely=0.2, anchor="center")
update_time(time_label)

def create_rounded_rectangle(canvas, x1, y1, x2, y2, radius=25, **kwargs):
    points = [
        (x1 + radius, y1),
        (x2 - radius, y1),
        (x2 - radius, y1),
        (x2, y1),
        (x2, y1 + radius),
        (x2, y2 - radius),
        (x2, y2 - radius),
        (x2, y2),
        (x2 - radius, y2),
        (x1 + radius, y2),
        (x1 + radius, y2),
        (x1, y2),
        (x1, y2 - radius),
        (x1, y1 + radius),
        (x1, y1 + radius),
        (x1, y1),
    ]
    return canvas.create_polygon(points, smooth=True, **kwargs)

# --> Frame for icons
icon_frame = tk.Frame(root, bg="#81c6c1")
icon_frame.place(relx=0.5, rely=0.5, anchor="center")

# --> Function to start chatbot interaction
def start_chatbot():
    
    for widget in icon_frame.winfo_children():
        widget.grid_forget()

    chat_frame = tk.Frame(root, bg="#81c6c1")
    chat_frame.place(relx=0.5, rely=0.5, anchor="center")

    chat_display = tk.Text(chat_frame, height=20, width=50, state="disabled", wrap="word", bg="#f7f7f7")
    chat_display.grid(row=0, column=0, padx=10, pady=10)

    user_input = tk.Entry(chat_frame, width=40)
    user_input.grid(row=1, column=0, padx=10, pady=5)

    # --> Function to process user input and generate bot response
    def process_input(event=None):  
        user_text = user_input.get().strip()
        if not user_text:
            return

        chat_display.config(state="normal")
        chat_display.insert("end", f"You: {user_text}\n")
        chat_display.config(state="disabled")

        if "joke" in user_text.lower():
            bot_response = tell_joke()
        else:
            sentiment, emotion = analyze_sentiment_and_emotion(user_text)
            bot_response = generate_response(user_text)

        chat_display.config(state="normal")
        chat_display.insert("end", f"Bot: {bot_response}\n")
        chat_display.config(state="disabled")
        user_input.delete(0, "end")

    user_input.bind("<Return>", process_input)  

    back_button = tk.Button(chat_frame, text="Back", command=lambda: show_main_page(chat_frame))
    back_button.grid(row=2, column=0, pady=10)


# --> Global variable to store the bot emoji label
bot_emoji_label = None

# --> Function to show the "About Bot" description and back button
def show_about():
    global bot_emoji_label
   
    time_label.place_forget()
    for widget in icon_frame.winfo_children():
        widget.grid_forget()

    bot_emoji_label = tk.Label(
        root,
        text="🤖",  
        font=("Arial", 30),
        fg="#2d2d2d",  
        bg="#81c6c1",  
    )
    bot_emoji_label.place(relx=0.5, rely=0.2, anchor="center")  

    description = """
    
\u2022 This bot is a friendly AI companion designed to chat,share jokes, and analyze emotions. It uses fine-tuned BlenderBot models and NLP pipelines to provide intelligent and engaging conversations.
\u2022 It can answer questions, provide support, and engage in friendly conversations."""

    description_label = tk.Label(
        icon_frame,
        text=description,
        font=("Arial", 14),
        fg="#2d2d2d",
        bg="#81c6c1",
        wraplength=350,
        anchor="w",  
        justify="left"  
    )
    description_label.grid(row=0, column=0, padx=15, pady=15)

    back_button_canvas = tk.Canvas(
        icon_frame,
        width=100,  
        height=40,  
        bg="#81c6c1",  
        highlightthickness=0
    )
    create_rounded_rectangle(
        back_button_canvas, 5, 5, 95, 35, radius=10, fill="#4e8f8e", outline="black"
    )
    back_button_canvas.create_text(
        50, 20, text="Back", font=("Arial", 12), fill="#2d2d2d"
    )
    back_button_canvas.grid(row=1, column=0, padx=15, pady=15)
    back_button_canvas.bind("<Button-1>", lambda event: show_main_page())

# --> Function to return to the main page with the buttons
def show_main_page(*args):
    global bot_emoji_label  

    if bot_emoji_label:
        bot_emoji_label.place_forget()  
        bot_emoji_label = None  

    for widget in root.winfo_children():
        if isinstance(widget, (tk.Frame, tk.Canvas)):
            widget.destroy()

    time_label.place(relx=0.5, rely=0.2, anchor="center")

    icons = [
        ("🤖", "Chat-Bot"),
        ("💬", "About Bot"), 
    ]

    global icon_frame
    icon_frame = tk.Frame(root, bg="#81c6c1")
    icon_frame.place(relx=0.5, rely=0.5, anchor="center")

    for idx, (emoji, name) in enumerate(icons):
        btn_canvas = tk.Canvas(icon_frame, width=220, height=60, bg="#81c6c1", highlightthickness=0)
        create_rounded_rectangle(
            btn_canvas, 5, 5, 215, 55, radius=20, fill="#4e8f8e", outline="black"
        )
        btn_canvas.create_text(
            110, 30, text=f"{emoji}  {name}", font=("Arial semibold", 16), fill="#2d2d2d"
        )
        btn_canvas.grid(row=idx, column=0, padx=15, pady=15)

        if name == "About Bot":
            btn_canvas.bind("<Button-1>", lambda event: show_about())
        else:
            btn_canvas.bind("<Button-1>", lambda event: start_chatbot())


show_main_page()

root.eval('tk::PlaceWindow . center')

root.mainloop()


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Device set to use cpu
